# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

print(len(full_data_rows_list))
print(full_data_rows_list[:2])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056
[['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91'], ['Manowar', 'Logged In', 'Jacob', 'M', '0', 'Klein', '247.562', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Shell Shock', '200', '1.54354E+12', '73']]


In [4]:
# checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance  local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Explaination

In our first query we need to filter the data by sessionId and Item in session columns, therefore they have to be in our primary key

**Query:** select artist,song,length from songs where sessionId=338 and itemInSession=4 

**Primary key:** Here our primary key is as follows (sessionId,itemInsession) where sessionId is the partition key and itemInsession is the clustering column. 

* It helps in faster retrieval of the song data from a given session.
* Partitioning the table by sessionId ensures that playlist data is stored by sessionId
* clustering column given as itemInsession makes sure that data is sorted by order of data.

#### CREATE TABLE

In [8]:
query= "CREATE table if not exists songs"
query=query + ("""(sessionId int, itemInSession int,artist text,song text, song_length float,PRIMARY KEY (sessionId,itemInSession))""")
try:
    session.execute(query)
except Exception as e:
    print(e)                   

#### INSERT data into TABLE

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs(sessionId,itemInSession,artist,song,song_length)"
        query =  query + "values (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### SELECT Statement to verify that the data have been inserted into each table

In [10]:
query = "SELECT * from songs limit 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.sessionId, row.Item_in_session, row.artist, row.songtitle,row.song_length)
    print(row[:])

(23, 0, 'Regina Spektor', 'The Calculation (Album Version)', 191.08526611328125)
(23, 1, 'Octopus Project', 'All Of The Champs That Ever Lived', 250.95791625976562)
(23, 2, 'Tegan And Sara', 'So Jealous', 180.06158447265625)
(23, 3, 'Dragonette', 'Okay Dolores', 153.39056396484375)
(23, 4, 'Lil Wayne / Eminem', 'Drop The World', 229.58975219726562)


#### Desired SELECT statement of the given query

In [11]:
query = "SELECT artist,song,song_length from songs where sessionId=338 and itemInSession=4 limit 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.sessionId, row.Item_in_session, row.artist, row.songtitle,row.song_length)
    print(row[:])

('Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Explaination

In our second query we need to filter the data by userId and sessionId, and they also have to be sorted by Item in session therefore they have to be in our primary key.  

**Query:** CREATE TABLE IF NOT EXISTS user_session (sessionId int, userId int, artist text, song text, firstName text, lastName text, itemInSession int, PRIMARY KEY ((sessionId, userId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)

**Primary key:** Here our primary key is ((sessionId,userId),itemInsession) where our partition key is (sessionId,userId) and clustering column is itemInsession

* Here the primary key given as userId and sessionId as we want to the data to be filtered by them 
* In order to sort the data by itemInsession it is given as clustering column which can give us the desired data for the given query.

#### CREATE TABLE

In [12]:
query= "CREATE table if not exists user_session"
query=query + ("""(user_id int,sessionid int,itemInSession int,artist text,songtitle text, first_name text,last_name text, PRIMARY KEY ((user_id,sessionId),itemInSession))\
with clustering order by (itemInSession ASC)""")
try:
    session.execute(query)
except Exception as e:
    print(e)              

#### INSERT data into TABLE

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session(user_id,sessionId,itemInSession,artist,songtitle,first_name,last_name)"
        query = query + "values (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

##### SELECT Statement to verify that the data have been inserted into each table

In [14]:
query = "SELECT * from user_session limit 5;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.sessionId, row.Item_in_session, row.artist, row.songtitle,row.song_length)
    print(row[:])

(58, 768, 0, 'System of a Down', 'Emily', 'Benson', 'Sad Statue')
(58, 768, 1, 'Ghostland Observatory', 'Emily', 'Benson', 'Stranger Lover')
(58, 768, 2, 'Evergreen Terrace', 'Emily', 'Benson', 'Zero')
(85, 776, 2, 'Deftones', 'Kinsley', 'Young', 'Head Up (LP Version)')
(85, 776, 3, 'The Notorious B.I.G.', 'Kinsley', 'Young', 'Playa Hater (Amended Version)')


In [15]:
query = "SELECT artist,songtitle,first_name,last_name from user_session where user_id=10 and sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.sessionId, row.Item_in_session, row.artist, row.songtitle,row.song_length)
    print(row[:])

('Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz')
('Three Drives', 'Greece 2000', 'Sylvie', 'Cruz')
('Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz')
('Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Explaination

Here our task is to give the details of the user who listened to a particular song 'All Hands against his own'

Therefore in our query, our primary key will comprise of song and also we should make sure that the result should not contain duplicates in the user details therefore userid will be added in the query.

**Primary key:** 

In our primary key, i.e.,((song),userId). Here,song will be our partition key and userId will be our clustering column. 

Here from the given query we could see that we want our data to retrieve by the given song.
* Therefore we will be giving song title as a primary key but in order to get a unique record, only song title is not sufficient. so clustering columns are required.
* user_id will be given as clustering column, possible choices for the clustering columns might be user first name and last name but there is a chance that user may share same first and last names. But user_id will be unique to each user, so it is a right choice for the clustering column.

#### CREATE TABLE

In [17]:
## Query 3: getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query= "CREATE table if not exists song_history"
query=query + "(song_title text,user_id int,first_name text,last_name text, PRIMARY KEY ((song_title),user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### INSERT data into TABLE

In [18]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history(song_title,user_id,first_name,last_name)"
        query = query + "values (%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

#### SELECT Statement to verify that the data have been inserted into each table

In [19]:
query = "SELECT * from song_history limit 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.sessionId, row.Item_in_session, row.artist, row.songtitle,row.song_length)
    print(row[:])

("Wonder What's Next", 49, 'Chloe', 'Cuevas')
("In The Dragon's Den", 49, 'Chloe', 'Cuevas')
('Too Tough (1994 Digital Remaster)', 44, 'Aleena', 'Kirby')
('Rio De Janeiro Blue (Album Version)', 49, 'Chloe', 'Cuevas')
('My Place', 15, 'Lily', 'Koch')


#### Desired SELECT statement of the given query

In [20]:
query = "SELECT first_name,last_name from song_history where song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.sessionId, row.Item_in_session, row.artist, row.songtitle,row.song_length)
    print(row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### DROPPING all tables

In [21]:
session.execute("DROP TABLE songs")
session.execute("DROP TABLE user_session")
session.execute("DROP TABLE song_history")

### Closing the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()